In [141]:
import os
import sys

import pandas as pd

In [142]:
table_dir = '/Users/eee/python/jra_ml/data/'
target_file = 'horse_result.csv'

output_dir = '/Users/eee/python/jra_ml/data/input'
output_file = 'input.csv'

table_path = os.path.join(table_dir, target_file)
output_path = os.path.join(output_dir, output_file)

In [143]:
df = pd.read_csv(table_path)
df.shape

(24642167, 29)

# 日本レースを抽出

In [144]:
place_urban = ['札幌', '函館', '福島', '新潟', '東京', '中山', '中京', '京都', '阪神', '小倉']
place_rural = ['帯広', '門別', '盛岡', '水沢', '浦和', '船橋', '大井', '川崎', '金沢', '笠松', '名古屋', '園田', '姫路', '高知', '佐賀']

#df = df[df['開催'].isin(place_urban + place_rural)]
re_patt = ''
for e in place_urban + place_rural:
    re_patt += e + '|'
re_patt = re_patt.rstrip('|')

df = df[df['開催'].str.contains(re_patt)]


# 欠損値処理

In [145]:
df = df.dropna(subset=['着順', '頭数', '騎手'])
df = df[df['着順'] != '失']
df = df[df['着順'] != '取']
df = df[df['着順'] != '中']
df = df[df['着順'] != '除']
df = df[df['頭数'] != 0]

df.head()

,horse_id,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
0,2019100855,2022/10/01,4中山8,晴,12.0,3歳以上2勝クラス,NaN,16.0,8.0,16,...,1.4,NaN,2-2,33.2-37.5,38.8,464(+2),NaN,NaN,ケイサンフリーゼ,NaN
1,2019100855,2022/09/10,4中山1,晴,6.0,3歳以上1勝クラス,NaN,13.0,6.0,8,...,-0.1,**,1-1,33.3-37.5,37.5,462(-6),NaN,NaN,(オンマガトオル),770.0
2,2019100855,2022/09/03,3新潟7,晴,8.0,3歳以上1勝クラス,NaN,15.0,5.0,9,...,1.9,**,2-2,33.9-36.9,38.5,468(0),NaN,NaN,ハイアムズビーチ,NaN
3,2019100855,2022/07/03,2福島2,晴,12.0,3歳以上1勝クラス,NaN,15.0,7.0,14,...,1.5,**,4-6,31.1-36.9,37.4,468(+12),NaN,NaN,ライヴサファイア,NaN
4,2019100855,2022/03/12,2中山5,晴,2.0,3歳未勝利,NaN,16.0,5.0,9,...,-0.3,**,1-1,34.4-37.5,37.5,456(-2),NaN,NaN,(オールフラッグ),520.0


In [146]:
df.isna().sum()

horse_id           0
日付                 0
開催                 0
天気                20
R                  0
レース名               0
映像          24232325
頭数                 0
枠番                 0
馬番                 0
オッズ             3062
人気                86
着順                 0
騎手                 0
斤量                 0
距離                 0
馬場                20
馬場指数         2186832
タイム                0
着差               119
ﾀｲﾑ指数        2192099
通過               603
ペース          3005545
上り             79756
馬体重                0
厩舎ｺﾒﾝﾄ      24232325
備考          24232325
勝ち馬(2着馬)          22
賞金          13206016
dtype: int64

# 着順、頭数のデータ型変更

In [147]:
df['着順'] = df['着順'].str.strip('(降)')
df['着順'] = df['着順'].str.strip('(再')
df['着順'] = df['着順'].astype(float).astype(int)
df['着順'].unique()

array([11,  1,  7, 10,  4,  2, 12,  8,  5,  9, 13,  3,  6, 15, 14, 16, 17,
       18])

In [148]:
df['頭数'] = df['頭数'].astype(int)
df['頭数'].unique()

array([16, 13, 15, 18, 17, 12, 14, 11, 10,  8,  9,  7,  6,  5,  4,  3])

In [149]:
df['着順/頭数'] = df['着順'] / df['頭数']
df.head()

,horse_id,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,...,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金,着順/頭数
0,2019100855,2022/10/01,4中山8,晴,12.0,3歳以上2勝クラス,NaN,16,8.0,16,...,NaN,2-2,33.2-37.5,38.8,464(+2),NaN,NaN,ケイサンフリーゼ,NaN,0.687500
1,2019100855,2022/09/10,4中山1,晴,6.0,3歳以上1勝クラス,NaN,13,6.0,8,...,**,1-1,33.3-37.5,37.5,462(-6),NaN,NaN,(オンマガトオル),770.0,0.076923
2,2019100855,2022/09/03,3新潟7,晴,8.0,3歳以上1勝クラス,NaN,15,5.0,9,...,**,2-2,33.9-36.9,38.5,468(0),NaN,NaN,ハイアムズビーチ,NaN,0.466667
3,2019100855,2022/07/03,2福島2,晴,12.0,3歳以上1勝クラス,NaN,15,7.0,14,...,**,4-6,31.1-36.9,37.4,468(+12),NaN,NaN,ライヴサファイア,NaN,0.666667
4,2019100855,2022/03/12,2中山5,晴,2.0,3歳未勝利,NaN,16,5.0,9,...,**,1-1,34.4-37.5,37.5,456(-2),NaN,NaN,(オールフラッグ),520.0,0.062500


# 頭数<=5は除外

In [150]:
df = df[df['頭数']>5]

# 学習用にデータ整形

In [151]:
temp_df = df.groupby('horse_id')
ids_per_group = temp_df.grouper.group_info[0]

In [152]:
df['ids_per_group'] = ids_per_group

In [153]:
count = 0
count_per_ids = []
for ids, v in enumerate(ids_per_group):
    if ids == 0 or ids_per_group[ids] != ids_per_group[ids-1]:
        count = 0
    else:
        count += 1
    count_per_ids += [count]

In [154]:
df['count_per_group'] = count_per_ids

In [171]:
pivot_df = df.pivot_table(values=['着順/頭数'], index=['horse_id'], columns=['count_per_group'], aggfunc='max')
pivot_df = pivot_df.iloc[:, :11]
pivot_df.reset_index(inplace=True)
pivot_df.columns = ['horse_id', '1試合前', '2試合前', '3試合前', '4試合前', '5試合前', '6試合前', '7試合前', '8試合前', '9試合前', '10試合前', '11試合前']
pivot_df = pivot_df.drop(columns=['1試合前'])

In [217]:
target_df = df.pivot_table(values=['着順/頭数'], index=['horse_id'], columns=['count_per_group'], aggfunc='max') 
target_df = target_df.iloc[:, :1]
target_df.reset_index(inplace=True)
target_df.columns = ['horse_id', 'target']
target_df = target_df.drop(columns=['horse_id'])
target_df[target_df['target']<0.2].head()
target_df = target_df.where(target_df['target'] <= 0.17, 0)
target_df = target_df.where(target_df['target'] == 0, 1)

In [218]:
pivot_df['target'] = target_df
pivot_df.to_csv(output_path, index=False)

In [221]:
pivot_df.head()

,horse_id,2試合前,3試合前,4試合前,5試合前,6試合前,7試合前,8試合前,9試合前,10試合前,11試合前,target
0,1998100265,0.875000,0.764706,0.769231,0.928571,0.588235,0.312500,0.062500,0.437500,0.090909,1.000000,0.0
1,1998105844,0.812500,0.812500,0.875000,0.866667,0.812500,0.888889,0.888889,1.000000,0.400000,0.375000,0.0
2,1999100207,0.888889,0.888889,1.000000,0.312500,0.562500,0.642857,0.125000,0.222222,0.555556,0.277778,0.0
3,1999104268,0.642857,0.666667,0.818182,1.000000,0.187500,0.625000,0.533333,0.923077,0.647059,0.333333,0.0
4,1999104764,0.400000,0.444444,0.272727,0.300000,0.333333,0.400000,0.100000,0.888889,0.125000,0.666667,0.0


In [222]:
print(pivot_df[pivot_df['target']==1].shape)
print(pivot_df[pivot_df['target']==0].shape)

(4550, 12)
(61950, 12)
